# En este código limpio la base de indicadores a nivel AGEB

Este notebook limpia la base de indicadores socioeconómicos a nivel AGEB de variables que no son necesarias. Además, debido a que la base originial está desagregada a nivel manzana, se utiliza solamente la observación con los datos agregados a nivel AGEB.

## Importacion de librerias y carga de archivo

In [1]:
import pandas as pd
import os

In [2]:
#Definimos la carpeta donde está el archivo
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_ageb"
archivo = "RESAGEBURB_09CSV20.csv"
ruta_completa = os.path.join(carpeta, archivo)

df_sample = pd.read_csv(ruta_completa, nrows = 5)
cols_to_use = ['ENTIDAD', 'MUN', 'LOC', 'AGEB', 'MZA', 'NOM_LOC', 'PHOG_IND', 'GRAPROES', 'POCUPADA', 'POBHOG', 'TVIVHAB', 'POBTOT' ]\
    
#Cargar solo las columnas necesarias
df_2020 = pd.read_csv(ruta_completa, usecols = cols_to_use)
df_2020

,ENTIDAD,MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,PHOG_IND,GRAPROES,POCUPADA,POBHOG,TVIVHAB
0,9,0,0,Total de la entidad,0000,0,9209944,289139,11.48,4985469,9159392,2757433
1,9,2,0,Total del municipio,0000,0,432205,7261,11.91,232719,431347,134204
2,9,2,1,Total de la localidad urbana,0000,0,432205,7261,11.91,232719,431347,134204
3,9,2,1,Total AGEB urbana,0010,0,3183,16,11.52,1532,3183,867
4,9,2,1,Azcapotzalco,0010,1,159,0,11.75,77,159,48
...,...,...,...,...,...,...,...,...,...,...,...,...
68936,9,17,1,Venustiano Carranza,1524,21,106,5,10.18,58,106,31
68937,9,17,1,Venustiano Carranza,1524,22,107,0,9.76,70,107,40
68938,9,17,1,Venustiano Carranza,1524,23,246,0,10.12,125,246,73
68939,9,17,1,Venustiano Carranza,1524,24,158,4,10.89,70,158,47


## Creación de la columna del ID de cada AGEB y eliminación de columnas inecesarias

In [3]:
# Crear la columna 'poligono_final' con formato: ENTIDAD(2) + MUN(3) + LOC(4) + AGEB(4)
df_2020["poligono_final"] = (
    df_2020["ENTIDAD"].astype(str).str.zfill(2) +
    df_2020["MUN"].astype(str).str.zfill(3) +
    df_2020["LOC"].astype(str).str.zfill(4) +
    df_2020["AGEB"].astype(str).str.zfill(4)
)

# Filtrar por "Total AGEB urbana"
df_2020 = df_2020[df_2020["NOM_LOC"] == "Total AGEB urbana"]

# Eliminar columnas que ya no se necesitan
df_2020_clean = df_2020.drop(columns=["ENTIDAD", "MUN", "LOC", "NOM_LOC", "MZA", "AGEB"])

# Nombres de columnas en minúsculas
df_2020_clean.columns = df_2020_clean.columns.str.lower()

#Pasar a numéricas las columnas
for col in df_2020_clean.columns:
    if col != "poligono_final":
        df_2020_clean[col] = pd.to_numeric(df_2020_clean[col], errors="coerce").fillna(0)

In [4]:
df_2020

,ENTIDAD,MUN,LOC,NOM_LOC,AGEB,MZA,POBTOT,PHOG_IND,GRAPROES,POCUPADA,POBHOG,TVIVHAB,poligono_final
3,9,2,1,Total AGEB urbana,0010,0,3183,16,11.52,1532,3183,867,0900200010010
30,9,2,1,Total AGEB urbana,0025,0,5593,69,10.73,2810,5593,1760,0900200010025
82,9,2,1,Total AGEB urbana,003A,0,4235,20,10.55,2080,4235,1190,090020001003A
116,9,2,1,Total AGEB urbana,0044,0,4768,58,11.07,2356,4768,1389,0900200010044
163,9,2,1,Total AGEB urbana,0097,0,2176,13,11.74,1058,2176,597,0900200010097
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68759,9,17,1,Total AGEB urbana,1488,0,1079,49,9.44,572,1079,284,0901700011488
68791,9,17,1,Total AGEB urbana,1492,0,900,28,7.5,425,900,242,0901700011492
68822,9,17,1,Total AGEB urbana,1505,0,2513,60,12.53,1324,2513,823,0901700011505
68870,9,17,1,Total AGEB urbana,151A,0,2248,34,12.03,1182,2248,717,090170001151A


In [5]:
df_2020_clean

,pobtot,phog_ind,graproes,pocupada,pobhog,tvivhab,poligono_final
3,3183,16.0,11.52,1532.0,3183.0,867,0900200010010
30,5593,69.0,10.73,2810.0,5593.0,1760,0900200010025
82,4235,20.0,10.55,2080.0,4235.0,1190,090020001003A
116,4768,58.0,11.07,2356.0,4768.0,1389,0900200010044
163,2176,13.0,11.74,1058.0,2176.0,597,0900200010097
...,...,...,...,...,...,...,...
68759,1079,49.0,9.44,572.0,1079.0,284,0901700011488
68791,900,28.0,7.50,425.0,900.0,242,0901700011492
68822,2513,60.0,12.53,1324.0,2513.0,823,0901700011505
68870,2248,34.0,12.03,1182.0,2248.0,717,090170001151A


## Exportar

In [6]:
carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

df_2020_clean.to_csv(os.path.join(carpeta_final, "controles_limpio.csv"), index = False, encoding='utf-8-sig')